In [ ]:
from pyspark.sql.types import *

In [ ]:
import pyspark.sql.functions as f

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql.types import StructField,IntegerType, FloatType, BooleanType ,StructType,StringType

In [ ]:
from pyspark.sql.dataframe import DataFrame

In [ ]:
from datetime import datetime

In [ ]:
from pyspark.sql import Row

In [ ]:
import os

In [ ]:
import json

In [ ]:
import csv

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import requests

In [ ]:
# definição do esquema da tabela temporária de clientes
camposClientes=[StructField('id',IntegerType(),True),
                StructField('nome',StringType(),True),
                StructField('cidade',StringType(),True),
                StructField('estado',StringType(),True),
                StructField('segmento',StringType(),True)
               ]
clientesStruct=StructType(fields=camposClientes)

In [ ]:
# definição do esquema da tabela temporária de planos
camposPlanos=[StructField('ID',IntegerType(),True),
             StructField('Plano',StringType(),True)]
planosStruct=StructType(fields=camposPlanos)

In [ ]:
# definição do esquema da tabela de pagamentos globais
camposPagamentosAno=[StructField('id',IntegerType(),True),
                  StructField('ano_pagamento',StringType(),True),
                  StructField('valorJan',FloatType(),True),
                  StructField('valorFev',FloatType(),True),
                  StructField('valorMar',FloatType(),True),
                  StructField('valorAbr',FloatType(),True),
                  StructField('valorMai',FloatType(),True),
                  StructField('valorJun',FloatType(),True),
                  StructField('valorJul',FloatType(),True),
                  StructField('valorAgo',FloatType(),True),
                  StructField('valorSet',FloatType(),True),
                  StructField('valorOut',FloatType(),True),
                  StructField('valorNov',FloatType(),True),
                  StructField('valorDez',FloatType(),True),
                  StructField('plano_id',IntegerType(),True)       ]
pagamentosAnoStruct=StructType(fields=camposPagamentosAno)


In [ ]:
# definição do esquema da tabela de pagamentos parcelados
camposPagamentosPar=[StructField('id',IntegerType(),True),
                  StructField('anomes_pagamento',StringType(),True),
                  StructField('valorAnoMes',FloatType(),True),
                  StructField('plano_id',IntegerType(),True)       ]
pagamentosParStruct=StructType(fields=camposPagamentosPar)


In [ ]:
# definição do esquema da tabela pagamentos 
camposPagamentosHj=[StructField('id',IntegerType(),True),
             StructField('data_pagamento',StringType(),True),
             StructField('valor',StringType(),True),
             StructField('plano_id',IntegerType(),True),
             StructField('qtde_meses_pagos',IntegerType(),True)
       ]
pagamentosHjStruct=StructType(fields=camposPagamentosHj)

In [ ]:
camposPagamentosGlobais=[StructField('id',IntegerType(),True),
                  StructField('ano_pagamento',StringType(),True),
                  StructField('valorJan',FloatType(),True),
                  StructField('valorFev',FloatType(),True),
                  StructField('valorMar',FloatType(),True),
                  StructField('valorAbr',FloatType(),True),
                  StructField('valorMai',FloatType(),True),
                  StructField('valorJun',FloatType(),True),
                  StructField('valorJul',FloatType(),True),
                  StructField('valorAgo',FloatType(),True),
                  StructField('valorSet',FloatType(),True),
                  StructField('valorOut',FloatType(),True),
                  StructField('valorNov',FloatType(),True),
                  StructField('valorDez',FloatType(),True),
                  StructField('plano_id',IntegerType(),True),  ]
pagamentosGlobaisStruct=StructType(fields=camposPagamentosGlobais)

In [ ]:
# criar a sessão de spark
spark = (
    SparkSession
    .builder
    .master("local")
    .appName("test-Cosmer")
    .config("spark.sql.shuffle.partitions", 13)
    .config("spark.driver.memory", "10g")
    .config("spark.sql.legacy.createHiveTableByDefault", "false")
    .config("spark.sql.debug.maxToStringFields", 1000000)
    .config("setLogLevel", "INFO")
    .config("spark.logConf", "true")
    .getOrCreate()
)
#spark.sparkContext.setLogLevel("WARN")  
    


In [ ]:
# ler api de clientes e criar o df de clientes
url = 'https://demo4461965.mockable.io/clientes'

response = requests.get(url=url)
#print('ini')
if response.status_code >= 200 and response.status_code <= 299:
    response.encoding = 'ISO-8859-1'
    #print('meio')
    tudo = response.text #.replace('[','').replace(']','')
    arqc = open("clientes.json", "w", encoding='ISO-8859-1')
    arqc.write(tudo)
    arqc.close()
    df_clientes = spark.read.format('json').schema(clientesStruct).load('clientes.json')
    df_clientes.write.format('parquet').partitionBy('id').mode('overwrite').save('/app/parquet/clientes/'+'df_clientes.parquet')


In [ ]:
# gerar df de planos
df_planos = spark.read.csv('planos.csv', header=True, sep=';',schema=planosStruct)


In [ ]:
# alterar nome de campos de df de planos
df_planos = df_planos.selectExpr("ID as plano_id", "Plano as Plano")

In [ ]:
# gerar parquet de planos
df_planos.write.format('parquet').mode('overwrite').save('/app/parquet/planos/'+'df_planos.parquet')

In [ ]:
# gerar df de pagamentos de hj
df_pagamentosHj = spark.read.csv('pagamentos*.csv', header=True, sep=';', schema=pagamentosHjStruct)


In [ ]:
# desmembra pagamentos para cada mês de cada ano/id de cliente
listaPagtos = []
for row in df_pagamentosHj.rdd.collect():
    qtd_meses = 0
    id = int(row['id'])
    str_date = row['data_pagamento']
    date = datetime.strptime(str_date, '%d/%m/%Y').date()
    #ano = f.colyear(row['data_pagamento'])
    #mes = f.month(row['data_pagamento'])
    #print(date, type(date))
    ano = str(date)[0:4]
    mes = str(date)[5:7]
    valor = float(row['valor'].replace('R$','').replace(',','.')) # float()
    meses = int(row['qtde_meses_pagos'])
    valorM = float(valor / meses)
    while qtd_meses < meses:
        if qtd_meses != 0:
            if mes == '12':
                mes = '01'
                ano = str(int(ano)+1)
            else:
                mes = str(int(mes)+1)
                if len(mes) == 1:
                    mes = '0'+mes
        anomes = ano + mes
        #print(anomes)
        #str_date = '2018-07-11'
        #date = datetime.strptime(str_date, '%Y-%m-%d').date()
        #print(date, type(date))
        dataP = date
        #print(row['id'], anomes , valorM , row['plano_id'], row['qtde_meses_pagos'], qtd_meses)
        linha = []
        linha.append(row['id'])
        linha.append(anomes)
        linha.append(valorM)
        linha.append(int(row['plano_id']))
        listaPagtos.append(linha)
        qtd_meses = qtd_meses + 1
        #print ('================')
del(df_pagamentosHj)


In [ ]:
# gerar rdd de pagamentos parciais
rdd = spark.sparkContext.parallelize(listaPagtos)
# criar df de pagamentos parciais
df_pagtosMensais=spark.createDataFrame(rdd, pagamentosParStruct)
#type(df_pagtosMensais)


In [ ]:
# excluir rdd
del(rdd)

In [ ]:
# construir o df da base de pagamentos mensais 
df_pagtosMensais = df_pagtosMensais \
                  .withColumn('year', f.col('anomes_pagamento').substr(1, 4)) \
                  .withColumn('month',f.col('anomes_pagamento').substr(5, 2))

In [ ]:
# separar a base de pagamentos mensais em meses
df_Jan = df_pagtosMensais.filter(df_pagtosMensais.month == '01').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorJan') 
df_Fev = df_pagtosMensais.filter(df_pagtosMensais.month == '02').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorFev')
df_Mar = df_pagtosMensais.filter(df_pagtosMensais.month == '03').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorMar')
df_Abr = df_pagtosMensais.filter(df_pagtosMensais.month == '04').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorAbr')
df_Mai = df_pagtosMensais.filter(df_pagtosMensais.month == '05').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorMai')
df_Jun = df_pagtosMensais.filter(df_pagtosMensais.month == '06').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorJun')
df_Jul = df_pagtosMensais.filter(df_pagtosMensais.month == '07').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorJul')
df_Ago = df_pagtosMensais.filter(df_pagtosMensais.month == '08').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorAgo')
df_Set = df_pagtosMensais.filter(df_pagtosMensais.month == '09').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorSet')
df_Out = df_pagtosMensais.filter(df_pagtosMensais.month == '10').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorOut')
df_Nov = df_pagtosMensais.filter(df_pagtosMensais.month == '11').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorNov')
df_Dez = df_pagtosMensais.filter(df_pagtosMensais.month == '12').groupby("id","year","plano_id").sum('valorAnoMes').withColumnRenamed('sum(valorAnoMes)','valorDez')



In [ ]:
# pegar horário de execução
horario = str(datetime.now())

In [ ]:
# retirar espaços do horário de execução para concatenar com o nome de pagamentos mensais
horario = horario.replace(' ','')

In [ ]:
# criar o parquet de pagamentos mensais com horario de execução para ser armazenado
df_pagtosMensais.write.format('parquet').save('/app/csv/processados/'+'pagamentosMensais'+horario)

In [ ]:
# criar a base de pagamentos gerais 
df_pagtoGeral = df_pagtosMensais \
                .select('id','year','plano_id') \
                .distinct() \
                .sort('id','year') 

In [ ]:
# excluir a base de df de pagamentos mensais
del(df_pagtosMensais)

In [ ]:
# fazer união com pagamentos mensais para trazer os dados para a base de pagamentos gerais
df_pagtoGeral = df_pagtoGeral.join(df_Jan, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Fev, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Mar, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Abr, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Mai, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Jun, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Jul, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Ago, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Set, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Out, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Nov, on=['id','year','plano_id'], how='left')
df_pagtoGeral = df_pagtoGeral.join(df_Dez, on=['id','year','plano_id'], how='left')


In [ ]:
# fazer união com Clientes para trazer os dados de clientes para a base de pagamentos gerais
#df_pagtoGeral = df_pagtoGeral.join(df_clientes, on=['id'], how='left')

In [ ]:
# fazer união com Plano para trazer os dados de clientes para a base de pagamentos gerais
#df_pagtoGeral = df_pagtoGeral.join(df_planos, on=['plano_id'], how='left')

In [ ]:
df_pagtoGeral.write.format('parquet').partitionBy('id').mode('overwrite').save('/app/parquet/pagamentos/df_pagtoGeral'+horario+'.parquet')


In [ ]:
# id|year|plano_id|valorJan|valorFev|valorMar|valorAbr|valorMai|valorJun|valorJul|valorAgo|valorSet|valorOut|valorNov|valorDez
# gravar parquet de pagamentos gerais
# df_pagtoGeral.write.format('parquet').partitionBy('id').mode('overwrite').save('/app/parquet/pagamentos/'+'df_pagtoGeral.parquet')
temPagamentos = df_pagtoGeral.count()
if temPagamentos > 0:
    if os.path.isfile('/app/parquet/pagamentosGerais/df_pagtoGeral.parquet'):
        df_pagtoGeralBase = spark.read.parquet("/app/parquet/pagamentosGerais/df_pagtoGeral.parquet")
        df_pagtoGeralBase = df_pagtoGeralBase \
                    .selectExpr("id as id", \
                                "year as year", \
                                "valorJan as valorJanBase", \
                                "valorFev as valorFevBase", \
                                "valorMar as valorMarBase", \
                                "valorAbr as valorAbrBase", \
                                "valorMai as valorMaiBase", \
                                "valorJun as valorJunBase", \
                                "valorAgo as valorAgoBase", \
                                "valorSet as valorSetBase", \
                                "valorOut as valorOutBase", \
                                "valorNov as valorNovBase", \
                                "valorMar as valorMarBase", \
                                "valorDez as valorDezBase", \
                                "plano_id as plano_id")
        df_pagtoGeralBase.fillna(0, inplace=True)
        df_full = df_pagtoGeralBase.join(df_pagtoGeral, on=['id', 'year', 'plano_id'], how='full_outer')
        #df_pagtoGeral.write.format('parquet').partitionBy('id').mode('overwrite').save('/app/parquet/pagamentos/'+'df_pagtoGeral.parquet')
        
        listaPagtosNew = []
        for row in df_full.rdd.collect():
            linhaNew = []
            linhaNew.append(row['id'])
            linhaNew.append(row['year'])
            linhaNew.append(row['plano_id'])
            if row['valorJan'] == 0:
                linhaNew.append(row['valorJanBase'])
            else:
                linhaNew.append(row['valorJan'])
            if row['valorFev'] == 0:
                linhaNew.append(row['valorFevBase'])
            else:
                linhaNew.append(row['valorFev'])
            if row['valorMar'] == 0:
                linhaNew.append(row['valorMarBase'])
            else:
                linhaNew.append(row['valorMar'])
            if row['valorAbr'] == 0:
                linhaNew.append(row['valorAbrBase'])
            else:
                linhaNew.append(row['valorAbr'])
            if row['valorMai'] == 0:
                linhaNew.append(row['valorMaiBase'])
            else:
                linhaNew.append(row['valorMai'])
            if row['valorJun'] == 0:
                linhaNew.append(row['valorJunBase'])
            else:
                linhaNew.append(row['valorJun'])
            if row['valorJul'] == 0:
                linhaNew.append(row['valorJulBase'])
            else:
                linhaNew.append(row['valorJul'])
            if row['valorAgo'] == 0:
                linhaNew.append(row['valorAgoBase'])
            else:
                linhaNew.append(row['valorAgo'])
            if row['valorSet'] == 0:
                linhaNew.append(row['valorSetBase'])
            else:
                linhaNew.append(row['valorSet'])
            if row['valorOut'] == 0:
                linhaNew.append(row['valorOutBase'])
            else:
                linhaNew.append(row['valorOut'])
            if row['valorNov'] == 0:
                linhaNew.append(row['valorNovBase'])
            else:
                linhaNew.append(row['valorNov'])
            if row['valorDez'] == 0:
                linhaNew.append(row['valorDezBase'])
            else:
                linhaNew.append(row['valorDez'])
                
            listaPagtosNew.append(linha)
        #print ('================')
        # gerar rdd de pagamentos parciais
        rdd = spark.sparkContext.parallelize(listaPagtosNew)
        # criar df de pagamentos parciais
        df_pagtoGeralTotal=spark.createDataFrame(rdd, pagamentosGlobaisStruct)
        #type(df_pagtosMensais)                                            
        df_pagtoGeralTotal.write.format('parquet').partitionBy('id').mode('overwrite').save('/app/parquet/pagamentosGerais/df_pagtoGeral.parquet')
    else:
        df_pagtoGeral.write.format('parquet').partitionBy('id').mode('overwrite').save('/app/parquet/pagamentosGerais/df_pagtoGeral.parquet')


In [ ]:
# alterar planilha de planos para processados
'''
os.chdir('.')
for f in os.listdir():
    if f == 'planos.csv':
        os.rename(f, '/app/csv/processados/' + f)
        print ('/app/csv/processados/' + f)
os.chdir('.')
'''


In [ ]:
# alterar planilha de pagamentos para processados
os.chdir('.')
for f in os.listdir():
    if f == 'pagamentos.csv':
        os.rename(f, '/app/csv/processados/' + f)
        print ('/app/csv/processados/' + f)
os.chdir('.')